In [1]:
import os
os.chdir('../../')

In [2]:
import data
import preprocess_utils.session2vec as sess2vec
import utils.sparsedf as sparsedf
from utils.df import MinMaxScaler
from utils.dataset import SequenceDatasetForBinaryClassification
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from IPython.display import display
pd.options.display.max_columns = None
tqdm.pandas()

Using TensorFlow backend.


In [3]:
from recommenders.recurrent.RNNBinaryClassificator import RNNBinaryClassificator

from numpy.linalg import norm as L2Norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, GRU, Embedding, Dropout, TimeDistributed
from keras.callbacks import EarlyStopping

In [4]:
mode = 'small'

## Build/load the model

In [5]:
weights = np.array([0.253, 0.747])
weights = 1/weights
wgt_sum = sum(weights)
weights = weights/wgt_sum

print(wgt_sum)
print(weights)

5.291257255636512
[0.747 0.253]


In [5]:
dataset = SequenceDatasetForBinaryClassification(f'dataset/preprocessed/cluster_recurrent/{mode}/dataset_binary_classification_p12')

In [7]:
model = RNNBinaryClassificator(dataset, input_shape=(dataset.rows_per_sample, 118), cell_type='gru', 
                                num_recurrent_units=64, num_recurrent_layers=2,
                                num_dense_layers=1, class_weights=[])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 12, 64)            35136     
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                9312      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
_________________________________________________________________
dropout_2 (Dropout)          (None, 1)                 0         
Total params: 45,537
Trainable params: 45,537
Non-trainable params: 0
_________________________________________________________________
None



In [31]:
x,y = dataset.load_Xtrain(), dataset.load_Ytrain()

X_train: (37664, 6, 42)
Y_train: (37664, 1)


In [22]:
x[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0

In [8]:
y

array([[0],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [9]:
len(y)

37664

In [7]:
np.sum(y, axis=0)

array([14185])

In [10]:
14185/37664

0.3766195836873407

## Train the model

In [30]:
import keras.backend as K

def softmax_loss(y_true, y_pred):
    #batch_dot = K.batch_dot(y_true, K.log(K.softmax(y_pred)))
    batch_dot = y_true * K.log(K.softmax(y_pred))
    return - K.sum( batch_dot, axis=-1 )

In [61]:
def softmax(x, axis=-1):
    y = np.exp(x - np.max(x, axis, keepdims=True))
    return y / np.sum(y, axis, keepdims=True)

In [6]:
m = Sequential()
#m.add( TimeDistributed(Dense(64), input_shape=(6,68)) )
m.add( GRU(64, input_shape=(6,42), recurrent_dropout=0.2, dropout=0.2, return_sequences=True) )
m.add( GRU(32, recurrent_dropout=0.2, dropout=0.2, return_sequences=False) )
m.add( Dense(32, activation='relu') )
m.add( Dropout(0.2) )
m.add( Dense(1, activation='sigmoid') )
m.add( Dropout(0.1) )

#sgd = keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
adam = keras.optimizers.Adam(lr=1e-3)
#rmsprop = keras.optimizers.RMSprop()
m.compile(adam, loss='binary_crossentropy', metrics=['accuracy'])
#m.compile(adam, loss=softmax_loss, metrics=['accuracy'])
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 6, 64)             20544     
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                9312      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
_________________________________________________________________
dropout_2 (Dropout)          (None, 1)                 0         
Total params: 30,945
Trainable params: 30,945
Non-trainable params: 0
_________________________________________________________________


In [33]:
x, y = shuffle(x, y)

In [35]:
m.fit(x=x, y=y, epochs=50, validation_split=0.2, batch_size=32, class_weight=dataset.get_class_weights(),
      callbacks=[EarlyStopping(patience=6, restore_best_weights=True)])

{0: 0.8020784530857362, 1: 1.3275995770179767}
Train on 30131 samples, validate on 7533 samples
Epoch 1/50
30131/30131 [==============================] - 25s 814us/step - loss: 1.2750 - acc: 0.7189 - val_loss: 0.5175 - val_acc: 0.7545
Epoch 2/50
30131/30131 [==============================] - 25s 825us/step - loss: 1.2892 - acc: 0.7188 - val_loss: 0.5095 - val_acc: 0.7555
Epoch 3/50
30131/30131 [==============================] - 25s 826us/step - loss: 1.2771 - acc: 0.7218 - val_loss: 0.5005 - val_acc: 0.7569
Epoch 4/50
30131/30131 [==============================] - 25s 829us/step - loss: 1.2810 - acc: 0.7228 - val_loss: 0.4959 - val_acc: 0.7589
Epoch 5/50
30131/30131 [==============================] - 25s 831us/step - loss: 1.2798 - acc: 0.7208 - val_loss: 0.5059 - val_acc: 0.7551
Epoch 6/50
30131/30131 [==============================] - 25s 836us/step - loss: 1.2333 - acc: 0.7264 - val_loss: 0.4963 - val_acc: 0.7583
Epoch 7/50
30131/30131 [==============================] - 26s 852us/st

In [10]:
model.model.save('gru.h5')

In [8]:
m = load_model('gru_binary.h5')

In [36]:
m.save('gru_binary_20_06_07595.h5')

In [7]:
m = load_model('gru_binary_20_06_07595.h5')

## Load saved model

In [6]:
model = RNNBinaryClassificator(dataset, input_shape=(dataset.rows_per_sample, 118), cell_type='gru', 
                                num_recurrent_units=64, num_recurrent_layers=2,
                                num_dense_layers=1, class_weights=[])
model.load('saved_models/rnn_GRU_2layers_64units_2dense_wgt_class_bin_84.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 12, 64)            35136     
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                9312      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
_________________________________________________________________
dropout_2 (Dropout)          (None, 1)                 0         
Total params: 45,537
Trainable params: 45,537
Non-trainable params: 0
_________________________________________________________________
None



## Use the model

In [20]:
m.predict(dataset.load_Xtest()[0])

array([[0.03439921],
       [0.62677795],
       [0.29741624],
       ...,
       [0.64165634],
       [0.06213458],
       [0.6602833 ]], dtype=float32)

In [8]:
#X_test_df = pd.read_csv(f'dataset/preprocessed/cluster_recurrent/{mode}/X_test.csv').set_index('orig_index')
#X_test_df = scale_dataframe(X_test_df, ['impression_price'])

target_indices = data.target_indices(mode, 'cluster_recurrent')
#target_indices = data.target_indices(mode, 'cluster_recurrent_up_to_len6')
#X_test_df.head(3)
target_indices[:10]

array([54066, 54071, 54108, 54184, 54211, 54343, 54397, 54606, 54609,
       54906])

In [9]:
recomendations = model.recommend_batch(target_indices)
if mode != 'full':
    model.compute_MRR(recomendations)

Loading full train_df, it will take a while..


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


TypeError: type object argument after * must be an iterable, not NoneType

In [7]:
model.evaluate()

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Scaler loaded!
X_test: (153130, 12, 118)
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Done!



              precision    recall  f1-score   support

      class1       0.84      0.77      0.80     95047
      class0       0.67      0.76      0.71     58083

   micro avg       0.77      0.77      0.77    153130
   macro avg       0.75      0.76      0.76    153130
weighted avg       0.77      0.77      0.77    153130



## RNN binary classificator GRU 2019-05-11_adam_e4_12_28_11

In [23]:
y_pred = m.predict_classes(xtest)
print(classification_report(y_true, y_pred, target_names=['1','0']))

              precision    recall  f1-score   support

           1       0.83      0.75      0.79      5724
           0       0.65      0.74      0.69      3522

   micro avg       0.75      0.75      0.75      9246
   macro avg       0.74      0.75      0.74      9246
weighted avg       0.76      0.75      0.75      9246



### Features: device, reference position, popularity, price, avg_price

#### 2x64 gru - 2 dense


In [ ]:
rec_df = pd.DataFrame(recomendations, columns=['id','recs'])
rec_df.head(10)

In [ ]:
data.train_df('full').loc[rec_df.id]

In [13]:
import out
out.create_sub(recomendations, submission_name='RNNclass_gru2x128_2dense_0617')

Exporting the sub to submissions/RNNclass_gru2x128_2dense_0617 18-38-09.csv...

submission created in submissions folder in 00m 03s


In [8]:
model.create_feature()

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Scaler saved!
X_train: (37664, 12, 118)
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Done!


,user_id,session_id,pred_rnn_binary
27778,0004IOZI7CKF,0146f7cb014ba,0.117823
464545,0008B0X0HC39,1b703eda68d85,0.846737
697007,000ZN9219G41,1086064207e4f,0.403117
884528,001OCXQ9PXET,362dc2be48010,0.405870
725484,001TEVEVUEBE,394f5ad9aa596,0.524890
450143,002IXCPKK20W,dcdc75a82f142,0.496907
368440,002O8H8GYKSQ,ab252cd2bafd3,0.154020
647663,002Z7VZFWM79,d4cc248f60752,0.574605
450147,003728BB2ALE,eaf4b65f13e8e,0.508730
203387,003CQCD8POV7,4c2ae42150f97,0.566502


## Evaluate the model

In [8]:
def compute_precision(predictions):
    train_df = data.train_df('full')

    target_indices, recs = zip(*predictions)
    target_indices = list(target_indices)
    
    df_target = train_df.loc[target_indices][['reference','impressions']]
    correct_clickouts = df_target.reference.values.astype('int')
    impressions = df_target.impressions.str.split('|').apply(lambda x: list(map(int, x)) ).values
    classes = [ impressions[i].index(correct_clickouts[i]) for i in range(len(impressions)) ]
    
    len_rec = len(recs)
    
    true_positive = np.zeros((25))
    count_per_class = np.zeros((25))

    for i in tqdm(range(len_rec)):
        cur_class = classes[i]
        count_per_class[cur_class] += 1

        correct_clickout = correct_clickouts[i]
        if correct_clickout == recs[i][0]:
            true_positive[cur_class] += 1
    
    print(f'precision per class: {true_positive / count_per_class}')
    
    print(f'true positive: {true_positive}')
    print(f'counts: {count_per_class}')
    return MRR

In [9]:
def compute_MRR_and_wrong(predictions):
    train_df = data.train_df('full')

    target_indices, recs = zip(*predictions)
    target_indices = list(target_indices)
    
    df_target = train_df.loc[target_indices][['reference','impressions']]
    correct_clickouts = df_target.reference.values.astype('int')
    impressions = df_target.impressions.str.split('|').apply(lambda x: list(map(int, x)) ).values
    classes = [ impressions[i].index(correct_clickouts[i]) for i in range(len(impressions)) ]
    
    len_rec = len(recs)
    
    false_positive = np.zeros((25))
    count_per_class = np.zeros((25))

    RR = 0
    for i in tqdm(range(len_rec)):
        cur_class = classes[i]
        count_per_class[cur_class] += 1

        correct_clickout = correct_clickouts[i]
        if correct_clickout != recs[i][0]:
            wrong_class = impressions[i].index(recs[i][0])
            false_positive[wrong_class] += 1
        
        if correct_clickout in predictions[i][1]:
            rank_pos = recs[i].index(correct_clickout) + 1
            if rank_pos <= 25:
                RR += 1 / rank_pos

    MRR = RR / len_rec
    print(f'MRR: {MRR}')
    
    print(f'wrong per class: {false_positive / count_per_class}')
    
    print(f'false positive: {false_positive}')
    print(f'counts: {count_per_class}')
    return MRR

In [10]:
x_test = dataset.load_Xtest()
recomendations = model.recommend_batch(target_indices)

caching df_full...


  0%|          | 337/153132 [00:00<03:12, 794.01it/s]

Done!


100%|██████████| 153132/153132 [01:00<00:00, 2520.79it/s]

prediction created !!!


In [28]:
if mode != 'full':
    compute_MRR_and_precision(recomendations)


MRR: 0.6316681353381653
precision per class: [0.9645003  0.26290209 0.26725267 0.25985178 0.25618254 0.25621415
 0.25706732 0.26542226 0.26653154 0.26792556 0.25610501 0.25658363
 0.26413718 0.25807754 0.27155963 0.25847458 0.2723192  0.26372443
 0.26710291 0.26889169 0.23430421 0.26831037 0.28317956 0.26984127
 0.26534191]
true positive: [56023.  4111.  2858.  2209.  1875.  1474.  1264.  1166.  1052.   979.
   839.   721.   724.   639.   592.   549.   546.   490.   449.   427.
   362.   370.   399.   391.   454.]
counts: [58085. 15637. 10694.  8501.  7319.  5753.  4917.  4393.  3947.  3654.
  3276.  2810.  2741.  2476.  2180.  2124.  2005.  1858.  1681.  1588.
  1545.  1379.  1409.  1449.  1711.]


In [10]:
if mode != 'full':
    compute_MRR_and_wrong(recomendations)

Loading full train_df, it will take a while..



MRR: 0.6316681353381653
wrong per class: [1.06974262 0.13813391 0.10510567 0.10387013 0.10848477 0.10203372
 0.1171446  0.10266333 0.10412972 0.11001642 0.1037851  0.10533808
 0.09157242 0.0957189  0.09633028 0.08333333 0.08678304 0.0818084
 0.09696609 0.07556675 0.07831715 0.07614213 0.07877928 0.0731539
 0.04792519]
false positive: [62136.  2160.  1124.   883.   794.   587.   576.   451.   411.   402.
   340.   296.   251.   237.   210.   177.   174.   152.   163.   120.
   121.   105.   111.   106.    82.]
counts: [58085. 15637. 10694.  8501.  7319.  5753.  4917.  4393.  3947.  3654.
  3276.  2810.  2741.  2476.  2180.  2124.  2005.  1858.  1681.  1588.
  1545.  1379.  1409.  1449.  1711.]


In [8]:
def create_feature_for_stacking(model, X, df):
    res_df = df.copy()
    res_df['score_rnn_bin'] = model.predict(X)
    return res_df

In [14]:
aa = pd.read_csv(f'dataset/preprocessed/cluster_recurrent/{mode}/dataset_binary_classification_p6/Y_train.csv')
aa.label.mean()

0.3766195836873407

In [15]:
aa.head()

,orig_index,user_id,session_id,timestamp,step,label
0,27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,0
1,464545,0008B0X0HC39,1b703eda68d85,1541445203,1,1
2,697007,000ZN9219G41,1086064207e4f,1541378829,11,1
3,884528,001OCXQ9PXET,362dc2be48010,1541102228,4,1
4,725484,001TEVEVUEBE,394f5ad9aa596,1541049172,1,0


### Dataset for stacking

In [10]:
create_feature_for_stacking(m, dataset.load_Xtrain(), pd.read_csv(f'dataset/preprocessed/cluster_recurrent/{mode}/dataset_binary_classification_p6/Y_train.csv'))

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


X_train: (673712, 6, 42)


,orig_index,user_id,session_id,timestamp,step,label,score_rnn_bin
0,8321598,0001VQMGUI65,466687f1875ee,1541076401,17,0,0.249901
1,12335024,0003QTCX5MJX,6e81f1bdeb75d,1541441472,4,1,0.446790
2,27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,0,0.191327
3,7333077,0004WCFRV3FB,b47e1ccca2b97,1541421814,2,1,0.384890
4,10577356,0006W0R5A5V8,85501a9f27263,1541385464,1,1,0.571049
5,464545,0008B0X0HC39,1b703eda68d85,1541445203,1,1,0.571049
6,13208141,0008BO33KUQ0,d4c5cc7f469d7,1541298200,12,0,0.233805
7,13933216,0008BO33KUQ0,eaae649fbd8c9,1541337472,4,0,0.273491
8,11769883,0008W02TLRNR,c39aa6a976e9b,1541305941,1,1,0.600197
9,9588794,000EKQEFMW6S,115f3370c7c43,1541277856,2,0,0.383297


In [11]:
_.set_index('orig_index').to_csv('dataset_stacking_binary_x.csv')

In [12]:
pd.DataFrame({'index':dataset.load_Xtest()[1], 'score_rnn_bin':m.predict(dataset.load_Xtest()[0]).flatten()})

X_test: (153130, 6, 42)


,index,score_rnn_bin
0,13395430,0.600197
1,9708521,0.417990
2,9066409,0.571049
3,11413083,0.146839
4,12030298,0.031020
5,10111509,0.153254
6,3409126,0.616958
7,10341252,0.600197
8,3656989,0.015318
9,14041108,0.571049


In [13]:
_.set_index('index').to_csv('dataset_stacking_binary_y.csv')